In [1]:
# from google.colab import drive

# drive.mount('/content/gdrive')
# root_path = '/D/Flatiron/Mod_5/Capstone'  #change dir to your project folder

In [2]:
# pip install pandas_datareader

In [3]:
# pip install nest_asyncio

In [4]:
# pip uninstall ipython

In [5]:
#import Libraries
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras import layers
import matplotlib.pyplot as plt

import praw
from psaw import PushshiftAPI
import config
import pandas as pd

import nltk
from nltk.corpus import stopwords
import string
from  nltk import FreqDist
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.collocations import *
# nltk.download('wordnet')
from nltk import word_tokenize, FreqDist

from spacy import displacy
import spacy
from tqdm import tqdm

tqdm.pandas()
from spacytextblob.spacytextblob import SpacyTextBlob

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')

import plotly.graph_objects as go
import plotly.express as px

import twint
import pandas as pd

import nest_asyncio 
nest_asyncio.apply()

from dateutil import rrule
import datetime as datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import time
start_time = time.time()

Using TensorFlow backend.


## Inputs:

In [6]:
currentDate = datetime.datetime.now()
q = 'TSLA'#ticker symbol
s = 'wallstreetbets' #Subreddit

In [7]:
currentDate.date()

datetime.date(2021, 6, 2)

In [8]:
Dates = []

for i in range(12):
    Dates.append((currentDate - relativedelta(weeks=i)).date())

In [9]:
Dates

[datetime.date(2021, 6, 2),
 datetime.date(2021, 5, 26),
 datetime.date(2021, 5, 19),
 datetime.date(2021, 5, 12),
 datetime.date(2021, 5, 5),
 datetime.date(2021, 4, 28),
 datetime.date(2021, 4, 21),
 datetime.date(2021, 4, 14),
 datetime.date(2021, 4, 7),
 datetime.date(2021, 3, 31),
 datetime.date(2021, 3, 24),
 datetime.date(2021, 3, 17)]

In [10]:
now = datetime.datetime.now()
hundredDaysLater = now - timedelta(days=300)

for dt in rrule.rrule(rrule.DAILY, dtstart=now, until=hundredDaysLater):
    print(dt)

In [11]:
r = praw.Reddit(
    client_id=config.reddit['client_id'],
    client_secret=config.reddit['client_secret'],
    username=config.reddit['username'],
    password=config.reddit['password'],
    user_agent='test'
)
api = PushshiftAPI(r)

## Webscraping:

In [12]:
#Created a function to streamline webscraping
def populate(currentDate, q, subreddit, limit):
#Reddit Posts    
    Dates = []
    gen = []

    for i in range(60):#Number of months
        Dates.append((currentDate - relativedelta(months=i)).date())
    
    for i in range(len(Dates)-1):

        
        gen.extend(list(api.search_submissions(after=Dates[i+1],
                            before=Dates[i],
                            q=q,
                            subreddit='wallstreetbets',
                            filter=['created', 'title'],
                            check_for_async=False,
                            limit=limit)))
    
#Turning gen list into a pandas dataframe    
    j = []

    for i in gen:
        j.append([i.title, 
#               i.author, 
#                   i.subreddit, 
#               i.score, 
                datetime.datetime.fromtimestamp(
                int(round(i.created))
                ).strftime('%Y-%m-%d')])#Returns the time the post was created in regular time instead of UNIX time])
        
    red_posts = pd.DataFrame(j,columns=['post', 
#                                 'author', 
#                                 'subreddit', 
#                                 'score', 
                                'date'])
    
    
#Tweets dataframe
    
    c = twint.Config()

    c.Search =q #search must contain ""
    c.Min_likes = 500 #min number of likes
    c.Until = str(currentDate.date()) #return tweets that were published before this date
    c.Since = str((currentDate - relativedelta(months=24)).date()) #return tweets published after this day(Months)
    c.Count = True
    c.Limit = 150000 # Limits the tweets to this number
    # c.Format = "Tweet id: {id} | Date: {date} | Username: {username} | Tweet: {tweet} | Mention: {mention}"
    c.Store_csv = True
    c.Output = 'Gamestop.csv'
    
    twint.run.Search(c)
    
    tweets = pd.read_csv('Gamestop.csv')
    tweets = tweets.rename(columns={"tweet": "post"})

    
    tweets = pd.concat([tweets['date'], tweets['post']], axis=1)
    
    
#combining reddit post and tweets    
    posts = pd.concat([red_posts, tweets])
    
    return posts
    

In [13]:
posts= populate(currentDate, q, s, 300)

C:\Users\eduar\anaconda3\envs\learn-env\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning:

Got non 200 code 429

C:\Users\eduar\anaconda3\envs\learn-env\lib\site-packages\psaw\PushshiftAPI.py:180: UserWarning:

Unable to connect to pushshift.io. Retrying after backoff.



1399443983915421701 2021-05-31 14:13:40 -0500 <jchybow> Had a dream last night that $TSLA shot up 40% in one day
1399431428689989635 2021-05-31 13:23:47 -0500 <themachinebuilt> You guys got it all wrong, Tesla and Micro-strategy didn’t put bitcoin on their balance sheet.   Bitcoin put $TSLA and $MSTR on it’s ledger. Bitcoin owns these companies now.   ♾️/21mil
1399231731543117826 2021-05-31 00:10:15 -0500 <adam82379116> .@elonmusk on Tesla Cybertruck: ‘The goal is to kick the most amount of ass possible with this truck’  $tsla  MISSION ACCOMPLISHED 😍  https://t.co/9vjADBaixn
1399190019747631109 2021-05-30 21:24:31 -0500 <ThaboAbbate> Drove past @elonmusk on the highway today 😂 $TSLA #Dogecoin  https://t.co/LxUeg0e34M
1399149836256124928 2021-05-30 18:44:50 -0500 <BahamaBen9> Let's play a game called "beat this portfolio over the next decade".  I pick 7 stocks that are equally weighted.  My list: 1. $SHOP 2. $SQ 3. $SE 4. $ROKU 5. $RDFN 6. $TWLO 7. $TSLA  You can replace one, multiple o

1395254536885788675 2021-05-20 00:46:18 -0500 <Travisdleslie> To compare:  $90,000 F-150 Lightning 0-60 4.5s 300 miles of range 10K towing  2K payload Pre Mapped HWY Blue Cruise 3rd party charging  $79,000 Tri Motor Cybertruck  0-60 2.9s 500+ miles or range 14k towing 3.5k payload FSD Largest charging network  $TSLA #TSLA $F @WholeMarsBlog
1395087256562503681 2021-05-19 13:41:36 -0500 <WhyTesla15> WOW $TSLA SEMI ON THE ROADS (CONVOY STYLE😜🤪) 😈😈😈    https://t.co/mAk6aRcm54
1395074445891543043 2021-05-19 12:50:41 -0500 <TeslaNY> Tesla Roadster at Petersen Automotive Museum - Video! 🏎🔋🔌  https://t.co/xo0NPBYIow $TSLA #Tesla #EV  https://t.co/brDDDGtRoA
1395070193458630659 2021-05-19 12:33:47 -0500 <bevedoni> If you are not buying this dip,  you are either so stupid or out of cash.   $TSLA
1395065639350509570 2021-05-19 12:15:42 -0500 <bull_trader_UK> Sooooooo. I can kinda leak some info. But not too much. That here in England there is a confirmed go ahead project I'm dealing with to start

1390250770549907457 2021-05-06 05:23:07 -0500 <DeItaone> TESLA IS CURRENTLY SEEING EXCEPTIONALLY STRONG DEMAND FOR ITS VEHICLES AS IT IS ALREADY SOLD OUT FOR THE QUARTER- ELECTREK  TESLA COMMUNICATED TO EMPLOYEES THAT PRODUCTION CAPACITY FOR Q2 IS ALREADY SOLD OUT WITH STILL ALMOST 2 MONTHS LEFT IN QTR - ELECTREK $TSLA
1390091666241966080 2021-05-05 18:50:54 -0500 <GerberKawasaki> He landed it! Congrats @elonmusk and @SpaceX ! $tsla #Tesla #ToTheMoon #tomars 🚀  https://t.co/qv9JReQZEG
1389972006913839105 2021-05-05 10:55:25 -0500 <bevedoni> RULES:   1. Respect Women 2. Work Hard 3. Own $DOGE and $TSLA  4. Drive Tesla  5. Use only Twitter  🥱
1389968198469095430 2021-05-05 10:40:17 -0500 <GerberKawasaki> Watch rocket engineer dealing with paparazzi. LOL. @elonmusk is classic. How things have changed in a few years. #tesla $tsla @SpaceX #SNL
1389954965981044745 2021-05-05 09:47:42 -0500 <TaylorOgan> $TSLA patent 👀  https://t.co/9IG7W42yCa
1389707925136973825 2021-05-04 17:26:03 -0500 <ste

1382324006880210945 2021-04-14 08:25:00 -0500 <wsbmod> On my deathbed my only regret will be that I didn't buy more TSLA calls
1382229034562580482 2021-04-14 02:07:37 -0500 <BMT094> Omg if this happens today:  $TSLA hits $800   #Bitcoin $68k 🚀  #dogecoin .20 cents  🐶
1382157733089013760 2021-04-13 21:24:17 -0500 <stevenmarkryan> 😂  Found in my comments. Love it.  $TSLA  https://t.co/SVFZoUJbyL
1382096020084035587 2021-04-13 17:19:03 -0500 <SawyerMerritt> $TSLA is now up 41.3% since the March 5th intraday low of $539.49 📈
1382080107817353222 2021-04-13 16:15:50 -0500 <thetradingchick> $tsla taxis!!! 🚖⚡️  https://t.co/ngJiitin0J
1382061147197739009 2021-04-13 15:00:29 -0500 <garyblack00> $TSLA 3 Gordons day!!
1382060677972389888 2021-04-13 14:58:37 -0500 <GerberKawasaki> My eight year old. “How many Gordons?”  Love it. $tsla #tesla
1382048440415293441 2021-04-13 14:10:00 -0500 <SawyerMerritt> ANNOYING PRICE SCREENSHOTS ARE BACK! $TSLA 🤣  https://t.co/CXVJNYPbzS
1382030321453916165 2021-0

...

1308397785134239745 2020-09-22 08:28:35 -0500 <vincent13031925> If you are a long term $TSLA investor, today will   BLOW  YOUR  FUCKING  MIND
1308161256654807040 2020-09-21 16:48:42 -0500 <zerohedge> TSLA down as much as one GM after hours
1307674942754811904 2020-09-20 08:36:16 -0500 <vincent13031925> After I bought my first Tesla in 2018, my $TSLA position increased 3X.  Owning the products allows you to deeply understand the company, way more than just reading news/info on internet.
1307529357951467531 2020-09-19 22:57:46 -0500 <StultusVox> Clip from China of a $TSLA Model 3, evidently equipped with the latest and greatest safety technology. My guess is the 4D dojo supercomputer recognizes that the pedestrian dummy is not a human and therefore sees no reason to perform evasive maneuvers.  https://t.co/R1mQqbxIy9
1306953794597482496 2020-09-18 08:50:41 -0500 <GerberKawasaki> I can tell you the team at tesla is working so hard. Elon is relentless and they have achieved great things. T

1291460563743145986 2020-08-06 14:46:07 -0500 <TeslaPodcast> $TSLA is extremely undervalued. Just for the record.
1289997787623337994 2020-08-02 13:53:34 -0500 <GerberKawasaki> Congrats @elonmusk and @SpaceX bringing humans back from space safely!!! Bob and Doug had an excellent adventure. #tesla $tsla
1289000568984203269 2020-07-30 19:50:58 -0500 <GerberKawasaki> Come on @elonmusk 5-1 just do it. The kids will go wild. $tsla #tesla
1288414554922405888 2020-07-29 05:02:22 -0500 <SamTwits> Latest pictures from Gigafactory Texas 1 week after breaking ground... 🤠💁‍♂️  #Tesla $TSLA @elonmusk  https://t.co/GUS0YHTMUt
1288315148818681856 2020-07-28 22:27:21 -0500 <CathieDWood> If successful, which we believe will be the case, a $TSLA ride-hailing service will lower the probability of our bear case substantially during the next five years, bolstering our base case at ~$7,000. We are encouraged that few analysts are modeling this possibility!
1288130135770587136 2020-07-28 10:12:11 -0500 <inum

1249026959998193664 2020-04-11 12:30:07 -0500 <vincent13031925> Biggest mistakes of the century:  From before,  Apple $AAPL is only a PC maker  Amazon $AMZN is only a bookstore   Google $GOOG $GOOGL is only a search engine   Netflix $NFLX is only a DVD rental company  Now,  Tesla $TSLA is only an automaker
1248770249643544576 2020-04-10 19:30:03 -0500 <vincent13031925> As a $TSLA shareholder, I don’t give a fuck of what Jim Chanos said.  https://t.co/89w6wZWnGX
1246926308073865216 2020-04-05 17:22:53 -0500 <facts_tesla> Just when you thought @latimes couldn't stoop any lower in its coverage of @Tesla and @elonmusk 😂😂😂  $TSLA  https://t.co/HDGw44H6hC
1246850058634616834 2020-04-05 12:19:53 -0500 <SamTwits> Nitrogen Oxide levels in Barcelona have fallen a staggering 83%, the majority of which has been attributed to an unprecedented drop in road traffic. More irrefutable evidence that ICE’s are the problem. Imagine the world we’d live in if we all drove electric cars? $TSLA #Tesla  https:

1214529546256601088 2020-01-07 06:49:43 -0500 <scottmelker> I made more money on a single $TSLA trade last year than on all of my crypto trading activities combined.
1214449340325875712 2020-01-07 01:31:00 -0500 <GerberKawasaki> Tesla short sellers are dead. Model Y line going up in China. China is going nuts for Tesla. This is amazing. Hahahahhahahaha great job @elonmusk - he also landed another rocket on a ship tonight. Time for bed.  $tsla
1214372150431191040 2020-01-06 20:24:17 -0500 <JayinShanghai> Elon Musk just arrived at Shanghai Gigafactory.  This time last year it was a mud pit.  Today, a year later, it has become a Tesla Gigafactory that can produce 3,000 Model 3s per week.  #Tesla #TeslaChina #ModelY #Model3 #GF3 #Gigafactory #特斯拉 #中国 $TSLA  https://t.co/efO7MxWSHK
1214291998586200064 2020-01-06 15:05:47 -0500 <vincent13031925> First time in the history, $TSLA is closed above $450 at $451.50  Congratulation to all #Tesla 🐮🐮🐮
1214064885518106625 2020-01-06 00:03:19 -0500 <vi

1138497852915703808 2019-06-11 12:26:55 -0500 <vincent13031925> Wow!! See what I found in here people ⚡️🙌🙌🙌🙌⚡️  Tesla Semi + Roadster 2 + Model Y  $TSLA #Tesla #TeslaSemi #Roadster2 #ModelY  https://t.co/8vfiMd8Auk
1138327887445090304 2019-06-11 01:11:32 -0500 <TeslaOptimist> Model 3 charging at 191 kW at IONITY charger. IONITY is a joint venture of BMW Group, Daimler AG, Ford Motor Company, and Volkswagen Group with Audi and Porsche" ... and the first production car to demonstrate this level of charging speed is a Tesla! :) $TSLA  https://t.co/DgAHqvxSEI
1136022818376654848 2019-06-04 16:32:00 -0500 <GerberKawasaki> Industries at risk when Tesla succeeds: Oil ICE cars Car dealers Rail and other energy transportation Auto parts auto service Gas stations Did I forget some? You wonder why there is so much FUD out there.  Netflix disrupted cable, Uber disrupted cab drivers, $Tsla $NFLX $UBER
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[+] Finished: Suc

In [15]:
posts

,post,date
0,$BBBY DD,2021-06-02
1,Here comes the sun turururu $RUN $TSLA,2021-06-02
2,$Run $Tsla f*** the suits,2021-06-02
3,B.A.R.S - The Ultimate Meme Portfolio 🚀🚀🚀,2021-06-01
4,(TSLA) vs (F),2021-06-01
...,...,...
4624,35% of Tesla shares r now shorted. 35% !. Almo...,2019-06-12
4625,"Just rolled over 5,000 miles this morning in T...",2019-06-12
4626,Wow!! See what I found in here people ⚡️🙌🙌🙌🙌⚡️...,2019-06-11
4627,Model 3 charging at 191 kW at IONITY charger. ...,2019-06-11


In [16]:
posts = posts.drop_duplicates(subset=['post'], ignore_index=True)

In [17]:
posts

,post,date
0,$BBBY DD,2021-06-02
1,Here comes the sun turururu $RUN $TSLA,2021-06-02
2,$Run $Tsla f*** the suits,2021-06-02
3,B.A.R.S - The Ultimate Meme Portfolio 🚀🚀🚀,2021-06-01
4,(TSLA) vs (F),2021-06-01
...,...,...
10188,Wow!! See what I found in here people ⚡️🙌🙌🙌🙌⚡️...,2019-06-11
10189,Model 3 charging at 191 kW at IONITY charger. ...,2019-06-11
10190,Industries at risk when Tesla succeeds: Oil IC...,2019-06-04
10191,Dear bitcoin miners. Please buy some solar and...,2021-05-12


## NLP:

In [18]:
#NLP Pipeline
nlp.component_names

['tok2vec',
 'tagger',
 'parser',
 'senter',
 'ner',
 'attribute_ruler',
 'lemmatizer',
 'spacytextblob']

In [19]:
posts['spacy'] = posts.post.progress_apply(lambda x: nlp(x))

100%|██████████| 10193/10193 [01:05<00:00, 154.62it/s]
C:\Users\eduar\anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [20]:
posts['sentiment'] = posts.spacy.progress_apply(lambda x: (x._.polarity))
posts['subjectivity'] = posts.spacy.progress_apply(lambda x: (x._.subjectivity))

100%|██████████| 10193/10193 [00:00<00:00, 201286.94it/s]
C:\Users\eduar\anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

100%|██████████| 10193/10193 [00:00<00:00, 302266.27it/s]
C:\Users\eduar\anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [22]:
posts.to_csv('static_dataset.csv')

## EDA:

In [ ]:
fig = px.scatter(posts, x='sentiment', y='subjectivity',
                hover_data=['post'])
fig.show()

In [ ]:
posts

In [ ]:
#Created two seperate dataframes for positive and negative posts
pos_list = posts[posts['sentiment'] > 0.0]
neg_list = posts[posts['sentiment'] < 0]

In [ ]:
pos_list

In [ ]:
neg_list

In [ ]:
pos_posts_concat = []
neg_posts_concat = []

for post in pos_list['post']:
    pos_posts_concat += post
for post in neg_list['post']:
    neg_posts_concat += post

In [ ]:
#created stop words list
stopwords_list = stopwords.words('english') + list(string.punctuation)
stopwords_list += ["''", '""', '...', '``', 'link', 'tesla', 'gamestop', "'", "'s", '’', 'retard', '[', ']', 'http', 'tsla',
                  'gme','tsla', 'http', 'https', "n't", 'fuck', 'stock', 'retards', 'amc', 'buy', 'people']

In [ ]:
#function for tokenized and lemmantized the post
def process_post(posts):
    lemmatizer = WordNetLemmatizer()
    tokens = nltk.word_tokenize(posts)
    stopwords_removed = [lemmatizer.lemmatize(token.lower()) for token in tokens if token.lower() not in stopwords_list]
    return stopwords_removed 

In [ ]:
#Fed boths postive and negative posts
pos_words = list(map(process_post, neg_list['post']))
neg_words = list(map(process_post, pos_list['post']))

In [ ]:
pos_words_list = []
for post in pos_words:
    for word in post:
        pos_words_list.append(word)
        
neg_words_list = []
for post in neg_words:
    for word in post:
        neg_words_list.append(word)

In [ ]:
pos_words_list

In [ ]:
pos_post_freqdist = FreqDist(pos_words_list)
pos_post_freqdist = pd.DataFrame(pos_post_freqdist.most_common(10), columns=['word', 'count'])

In [ ]:
#Graphing most common positive posts
fig1 = px.bar(pos_post_freqdist, y='count', x='word', 
              hover_data=['count'],
              color='count',
              title=' Most frequent positive')
fig1.show()

In [ ]:
neg_post_freqdist = FreqDist(neg_words_list)
neg_post_freqdist = pd.DataFrame(neg_post_freqdist.most_common(10), columns=['word', 'count'])

In [ ]:
#Graphing most negative posts
fig1 = px.bar(neg_post_freqdist, y='count', x='word', 
              hover_data=['count'],
              color='count',
              title=' Most frequent Negative')
fig1.show()

In [ ]:
#grouping the posts by day and then returning average values for sentiment and subjectivity
final_posts = posts.groupby(['date']).mean().sort_values(by='date', ascending=False)

In [ ]:
final_posts

In [ ]:
final_posts.index.names = ['Date']

In [ ]:
final_posts

In [ ]:
from matplotlib.pyplot import figure

In [ ]:
#Get the stock quote
tick = web.DataReader(q, data_source='yahoo', start=str((currentDate - relativedelta(months=12)).date()), end=str(currentDate.date()))
tick

In [ ]:
tick.shape

## Merging Posts:

In [ ]:
df= tick.join(final_posts)

In [ ]:
#Get number of rows and columns
df.shape

In [ ]:
df.isna()

In [ ]:
#Filled NaN values with the average of the columns
df = df.fillna(df.mean())

In [ ]:
df = pd.DataFrame(df)

In [ ]:
df['Target'] = np.append(df['Close'].iloc[1:].values, [np.nan])

In [ ]:
df

In [ ]:
df = df.dropna()

In [ ]:
df

In [ ]:
type(df)

In [ ]:
df

In [ ]:
#Visualize The data
fig,ax = plt.subplots(figsize=(20, 8))

ax.set_title('Closing Price and Sentiment History')
ax.plot(df['Close'], color='red')
ax.set_xlabel('Date', fontsize=18)
ax.set_ylabel('Closing Price USD ($)',fontsize=18)

ax2=ax.twinx()
ax2.plot(df['sentiment'], color='Blue', marker="o")
ax2.set_xlabel('Date', fontsize=18)
ax2.set_ylabel('Sentiment',fontsize=18)
ax.legend()